In [24]:
#import libraries
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda,Compose
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [25]:
training_data = datasets.FashionMNIST(
root="data",
train=True,
download=True,
transform=ToTensor()

)

In [26]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()

)

In [27]:
batch_size = 64

#create data loaders
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print("Shape of X [N, C, H, W] : ", X.shape)
    print("Shape of y : ", y.shape,y.dtype)
    break

Shape of X [N, C, H, W] :  torch.Size([64, 1, 28, 28])
Shape of y :  torch.Size([64]) torch.int64


In [28]:
#creating models
device = "cuda" if torch.cuda.is_available() else "cpu"
print('using {} device'.format(device))

using cpu device


In [43]:
#define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )

    def forward(self,X):
        x = self.flatten(X)
        logits = self.linear_relu_stack(x)
        return logits
        


In [44]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [45]:
loss_fn = nn.CrossEntropyLoss()
lr = 1e-3
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

In [46]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        #compute prediction error
        pred = model(X)
        loss  = loss_fn(pred,y)

        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss,current = loss.item(),batch *len(X)
            print(f'loss: {loss:>7f}[{current:>5d}/{size:>5d}]')

In [47]:
def test(dataloader,model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /=size
    correct /= size
    print(f'test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [48]:
epochs = 1
for t in range(epochs):
    print(f'Epoch{t+1}\n---------------')
    train(train_dataloader,model,loss_fn,optimizer)
    #test(test_dataloader,model)
print("Done!")

Epoch1
---------------
loss: 2.303719[    0/60000]
loss: 2.293729[ 6400/60000]
loss: 2.276447[12800/60000]
loss: 2.272671[19200/60000]
loss: 2.272403[25600/60000]
loss: 2.257005[32000/60000]
loss: 2.252263[38400/60000]
loss: 2.232702[44800/60000]
loss: 2.247400[51200/60000]
loss: 2.242161[57600/60000]
Done!


In [49]:
torch.save(model.state_dict(),"data/model.pth")
print("saved pytorch model state to model.pth")

saved pytorch model state to model.pth


In [50]:
#loading models
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

In [52]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankel boot"
]


model.eval()
x,y = test_data[0][0],test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted,actual = classes[pred[0].argmax(0)],classes[y]
    print(f'predicted: "{predicted}",Actual: "{actual}"')


predicted: "Ankel boot",Actual: "Ankel boot"
